In [28]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
dfd=pd.read_csv("day.csv")
dfh=pd.read_csv("hour.csv")
dfd=dfd.rename(columns={"dteday":"date","yr":"year","atemp":"ftemp","mnth":"month","weathersit":"weather","weekday":"day","cnt":"count"})
dfh=dfh.rename(columns={"dteday":"date","yr":"year","atemp":"ftemp","mnth":"month","hr":"hour","weathersit":"weather","weekday":"day","cnt":"count"})
dfd["date"]=pd.to_datetime(dfd["date"])
dfh["date"]=pd.to_datetime(dfh["date"])
#No missing values

In [33]:
dfh.isnull().sum().sum()

0